Model Documentation
Overview

This documentation provides details on a handwritten character recognition model developed using TensorFlow and Keras. The project is split into three main scripts: create_model.py, dataset.py, and improve_model.py. The purpose of the model is to recognize characters from the A-Z Handwritten Data dataset.

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from dataset import *

1. dataset.py

1.1. split_dataset

Purpose:

Splits the A-Z Handwritten Data dataset into training and testing sets.

Functionality
Reads the dataset from 'main_dataset/A_Z Handwritten Data.csv'.
Splits the dataset into features (x) and labels (y).
Performs a train-test split (80% training, 20% testing).

In [2]:
def split_dataset(): # returns in-order: x_train, x_test, y_train, y_test
    csv_file= pd.read_csv('main_dataset/A_Z Handwritten Data.csv').astype('float32')
        
    dataset = pd.DataFrame(csv_file)
    
    x = dataset.drop('0', axis = 1)
    y = dataset['0']
    
    return train_test_split(x, y, test_size = 0.2) # 20% train set

1.2. dataset_train_test_to_csv

Purpose:

Creates CSV files for training and testing sets.

Functionality
Calls split_dataset to get x_train, y_train, x_test, y_test.
Creates CSV files for each set in the 'subdataset' directory.

In [3]:
def dataset_to_csv(x1,y1,x2,y2): # creates csv for x_train, x_test, y_train, y_test
    
    x_train, x_test, y_train, y_test = split_dataset()
    
    print('\nCreating Csv 1/4')
    x_train.head(x1).to_csv('subdataset/x_train.csv', index=False, header=False)
    
    print('\nCreating Csv 2/4')
    y_train.head(y1).to_csv('subdataset/y_train.csv', index=False, header=False)
    
    print('\nCreating Csv 3/4')
    x_test.head(x2).to_csv('subdataset/x_test.csv', index=False, header=False)
    
    print('\nCreating Csv 4/4')
    y_test.head(y2).to_csv('subdataset/y_test.csv', index=False, header=False)
    
    print('\n-- Done --')

1.3. verify_all_datasets

Purpose:

Loads all datasets and prints their shapes.

Functionality
Calls dataset loading functions and prints shapes for x_train, y_train, x_test, and y_test.

In [4]:
def verify_all_datasets(): # loads all datasets, still kept just in-case.
    
    x_train = load_x_train()
    y_train = load_y_train()
    x_test= load_x_test()
    y_test = load_y_test()
    
    print(f'\nx_train shape: {x_train.shape}')
    print(f'\ny_train shape: {y_train.shape}')
    print(f'\nx_test shape: {x_test.shape}')
    print(f'\ny_test shape: {y_test.shape}')
    

1.4. load_x_train, load_y_train, load_x_test, load_y_test

Purpose:

Loads specific datasets from CSV files.

Functionality
Reads the corresponding CSV files and returns the datasets.

In [5]:
def load_x_train():
    return pd.read_csv('subdataset/x_train.csv', header=None).astype('float32')

def load_y_train():
    return pd.read_csv('subdataset/y_train.csv', header=None).astype('float32')

def load_x_test():
    return pd.read_csv('subdataset/x_test.csv', header=None).astype('float32')

def load_y_test():
    return pd.read_csv('subdataset/y_test.csv', header=None).astype('float32')

1.5. limit_train_count

Purpose:

Limits the number of training iterations based on user input.

Functionality
Takes a user-input count and ensures it is within specified bounds (min: 0, max: 10,000).

In [6]:
def limit_train_count(trainCount, min, max):
    if(trainCount > max): return max
    elif(trainCount < min): return min
    return trainCount

2. create_model.py

2.1. Script Overview

Purpose:

Creates and trains a handwritten character recognition model using a specified architecture.

Key Components:

Sequential model with input layer, two hidden layers, and output layer.
Dense layers with ReLU activation and L2 regularization.
Training with Adam optimizer and Sparse Categorical Crossentropy loss.

In [7]:
import tensorflow as tf;
from dataset import load_x_train, load_y_train, limit_train_count
from keras.models import Sequential
from keras.layers import Dense


2023-12-13 14:21:27.070613: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-13 14:21:27.142181: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-13 14:21:27.737723: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-13 14:21:27.741317: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-13 14:21:28.977719: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

2.2. Script Flow

Loading Dataset

Calls load_x_train and load_y_train to load training data.

Creating Model

    -Defines a Sequential model with input layer, two hidden layers, and output layer.
    -Uses ReLU activation and L2 regularization in hidden layers.

Training Model

    -Compiles the model with Adam optimizer and Sparse Categorical Crossentropy loss.
    -Takes user input for initial training iterations.
    -Fits the model to the training data with validation split.

Saving Model

    -Takes user input for a new model name.
    -Saves the trained model in the 'model' directory.

In [10]:
print('-- Loading dataset...')

x_train = load_x_train()
y_train = load_y_train()

print(f'\nx_train shape: {x_train.shape}')
print(f'\ny_train shape: {y_train.shape}')

print('\n-- Dataset prepared!')

print('\n-- Creating Model')

model = Sequential(
    [               
        tf.keras.Input(shape=(784,)),
        
        Dense(126, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.2)),
        Dense(64, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.2)),
        Dense(26, activation='softmax')
        
    ], name = "new_model" 
) # recommended fit = 200

print('\n-- Model Created!')

print('\n-- Training model\n')

trainCount = int(input('Initial Training Iterations (min: 1, max: 1000): '))

model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    metrics=['accuracy'],
)

model.fit(x_train, y_train, validation_split = 0.1, epochs=limit_train_count(trainCount,0,1000))


print('\n-- Training end, saving model\n')
modelName = str(input('Enter a new model name: '))
model.save(f'./model/{modelName}.keras')

-- Loading dataset...

x_train shape: (20000, 784)

y_train shape: (20000, 1)

-- Dataset prepared!

-- Creating Model

-- Model Created!

-- Training model

Epoch 1/100
563/563 [==============================] - 2s 3ms/step - loss: 65.5326 - accuracy: 0.2448 - val_loss: 41.0778 - val_accuracy: 0.2835
Epoch 2/100
563/563 [==============================] - 1s 3ms/step - loss: 36.9336 - accuracy: 0.3060 - val_loss: 33.6931 - val_accuracy: 0.3535
Epoch 3/100
563/563 [==============================] - 1s 2ms/step - loss: 30.9811 - accuracy: 0.3902 - val_loss: 28.3268 - val_accuracy: 0.4455
Epoch 4/100
563/563 [==============================] - 1s 2ms/step - loss: 25.7767 - accuracy: 0.4799 - val_loss: 23.2422 - val_accuracy: 0.5400
Epoch 5/100
563/563 [==============================] - 1s 2ms/step - loss: 20.8853 - accuracy: 0.5423 - val_loss: 18.5777 - val_accuracy: 0.5735
Epoch 6/100
563/563 [==============================] - 1s 2ms/step - loss: 16.4867 - accuracy: 0.5888 - val_loss: 14.

KeyboardInterrupt: 

3. improve_model.py

3.1. Script Overview

Purpose:

Loads an existing model and performs additional training to improve performance.

Key Components:

Loads a pre-existing model using tf.keras.models.load_model.
Takes user input for additional training iterations.

In [ ]:
import tensorflow as tf;
from dataset import load_x_train, load_y_train, limit_train_count

3.2. Script Flow

Loading Dataset

    -Calls load_x_train and load_y_train to load training data.

Loading Existing Model

    -Takes user input for an existing model name.
    -Attempts to load the model; exits if not found.

Additional Training

    -Takes user input for additional training iterations.
    -Performs additional training on the loaded model.

Updating and Saving Model

    -Takes user input for a new model name.
    -Saves the updated model in the 'model' directory.

In [ ]:
print('-- Loading dataset...')

x_train = load_x_train()
y_train = load_y_train()

print(f'\nx_train shape: {x_train.shape}')
print(f'\ny_train shape: {y_train.shape}')

print('\n-- Dataset prepared!')

print('\n-- Training model\n') 


modelName = str(input('Enter an existing model name: '))
try:
    modelLoad = tf.keras.models.load_model(f'model/{modelName}.keras')
except:
    print('Model not found!')
    exit()

trainCount = int(input('Training Iterations (min: 0, max: 1000): '))

modelLoad.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    metrics=['accuracy'],
)

modelLoad.fit(
    x_train, y_train, validation_split = 0.1,
    epochs=limit_train_count(trainCount,0,1000) # basically an input with min and max limit
)


print('\n-- Training end, updating model\n')
modelLoad.save(f'./model/{modelName}.keras')

Conclusion

The developed model provides a framework for character recognition, and the scripts facilitate dataset handling, model creation, and improvement. Users can leverage these scripts to experiment with different architectures, training iterations, and datasets for their specific requirements.